In [2]:
# # Установка XGBoost
# !pip install xgboost

# # Установка CatBoost
# !pip install catboost

# # Установка TensorFlow
# !pip install tensorflow

# # Установка Scikeras 
# !pip install scikeras

In [3]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from tensorflow import keras
from tensorflow.keras import layers as L
from scikeras.wrappers import KerasRegressor

In [4]:
# Чтение DataFrame
df_models = pd.read_csv("данные для моделей.csv")
print("DataFrame 'данные для моделей.csv' успешно загружен.")
  

DataFrame 'данные для моделей.csv' успешно загружен.


In [5]:
#Подготовка данных для IC50
# Целевая переменная для IC50
target_ic50 = 'log_IC50, mM'

# Признаки (X) и целевая переменная (y)
# Исключаем другие целевые переменные из признаков
X = df_models.drop(columns=['log_IC50, mM', 'log_CC50, mM', 'log_SI'])
y = df_models[target_ic50]

# Разделение на обучающую и тестовую выборки
# random_state для воспроизводимости
X_train_ic50, X_test_ic50, y_train_ic50, y_test_ic50 = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n--- Подготовка данных для IC50 ---")
print(f"Размер обучающей выборки (X_train_ic50): {X_train_ic50.shape}")
print(f"Размер тестовой выборки (X_test_ic50): {X_test_ic50.shape}")


--- Подготовка данных для IC50 ---
Размер обучающей выборки (X_train_ic50): (773, 68)
Размер тестовой выборки (X_test_ic50): (194, 68)


In [6]:
# 3. Настройка Pipeline и GridSearchCV

# Метрики для GridSearchCV. 'neg_' означает, что GridSearchCV стремится максимизировать метрику,
scoring_metrics = {
    'MSE': 'neg_mean_squared_error',
    'R2': 'r2',
    'MAE': 'neg_mean_absolute_error'
}

In [7]:
# Модель 1: Линейная Регрессия 
print("\n===== Линейная Регрессия для IC50 =====")
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

param_grid_lr = {} # Для LinearRegression нет гиперпараметров 

grid_search_lr = GridSearchCV(
    pipeline_lr,
    param_grid_lr,
    cv=5, # 5-кратная кросс-валидация
    scoring=scoring_metrics,
    refit='R2', # Оптимизировать по R2
    n_jobs=-1, # Использовать все доступные ядра
    verbose=1 # Выводить прогресс
)

grid_search_lr.fit(X_train_ic50, y_train_ic50)

print("Лучшие параметры для Линейной Регрессии:", grid_search_lr.best_params_)
print("Лучший R2 на кросс-валидации:", grid_search_lr.best_score_)

# Оценка на тестовой выборке
y_pred_lr = grid_search_lr.predict(X_test_ic50)
mse_lr = mean_squared_error(y_test_ic50, y_pred_lr)
r2_lr = r2_score(y_test_ic50, y_pred_lr)
mae_lr = mean_absolute_error(y_test_ic50, y_pred_lr)

print("\nМетрики на тестовой выборке (Линейная Регрессия):")
print(f"MSE: {mse_lr:.4f}")
print(f"MAE: {mae_lr:.4f}")
print(f"R2: {r2_lr:.4f}")


===== Линейная Регрессия для IC50 =====
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Лучшие параметры для Линейной Регрессии: {}
Лучший R2 на кросс-валидации: 0.2813431323334865

Метрики на тестовой выборке (Линейная Регрессия):
MSE: 0.5935
MAE: 0.6009
R2: 0.2753


In [8]:
# Модель 2: Random Forest Регрессор 
print("\n===== Random Forest Регрессор для IC50 =====")
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Гиперпараметры для настройки Random Forest
param_grid_rf = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_features': [0.6, 0.8, 1.0],
    'regressor__min_samples_leaf': [5, 10]
}

grid_search_rf = GridSearchCV(
    pipeline_rf,
    param_grid_rf,
    cv=5,
    scoring=scoring_metrics,
    refit='R2',
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X_train_ic50, y_train_ic50)

print("Лучшие параметры для Random Forest:", grid_search_rf.best_params_)
print("Лучший R2 на кросс-валидации:", grid_search_rf.best_score_)

# Оценка на тестовой выборке
y_pred_rf = grid_search_rf.predict(X_test_ic50)
mse_rf = mean_squared_error(y_test_ic50, y_pred_rf)
r2_rf = r2_score(y_test_ic50, y_pred_rf)
mae_rf = mean_absolute_error(y_test_ic50, y_pred_rf)

print("\nМетрики на тестовой выборке (Random Forest):")
print(f"MSE: {mse_rf:.4f}")
print(f"MAE: {mae_rf:.4f}")
print(f"R2: {r2_rf:.4f}")


===== Random Forest Регрессор для IC50 =====
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Лучшие параметры для Random Forest: {'regressor__max_features': 0.6, 'regressor__min_samples_leaf': 5, 'regressor__n_estimators': 200}
Лучший R2 на кросс-валидации: 0.4292621189253373

Метрики на тестовой выборке (Random Forest):
MSE: 0.4616
MAE: 0.5325
R2: 0.4364


In [9]:
# Модель 3: LightGBM Регрессор 
print("\n===== LightGBM Регрессор для IC50 =====")
pipeline_lgbm = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LGBMRegressor(random_state=42))
])

# Гиперпараметры для настройки LightGBM
param_grid_lgbm = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__num_leaves': [31, 63],
    'regressor__reg_alpha': [0.1, 0.5],
    'regressor__reg_lambda': [0.1, 0.5]
}

grid_search_lgbm = GridSearchCV(
    pipeline_lgbm,
    param_grid_lgbm,
    cv=5,
    scoring=scoring_metrics,
    refit='R2',
    n_jobs=-1,
    verbose=1
)

grid_search_lgbm.fit(X_train_ic50, y_train_ic50)

print("Лучшие параметры для LightGBM:", grid_search_lgbm.best_params_)
print("Лучший R2 на кросс-валидации:", grid_search_lgbm.best_score_)

# Оценка на тестовой выборке
y_pred_lgbm = grid_search_lgbm.predict(X_test_ic50)
mse_lgbm = mean_squared_error(y_test_ic50, y_pred_lgbm)
r2_lgbm = r2_score(y_test_ic50, y_pred_lgbm)
mae_lgbm = mean_absolute_error(y_test_ic50, y_pred_lgbm)

print("\nМетрики на тестовой выборке (LightGBM):")
print(f"MSE: {mse_lgbm:.4f}")
print(f"MAE: {mae_lgbm:.4f}")
print(f"R2: {r2_lgbm:.4f}")


===== LightGBM Регрессор для IC50 =====
Fitting 5 folds for each of 72 candidates, totalling 360 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001142 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9262
[LightGBM] [Info] Number of data points in the train set: 773, number of used features: 62
[LightGBM] [Info] Start training from score 1.653887
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

In [10]:
# Модель 4: XGBoost Regressor
print("\n===== XGBoost Регрессор для IC50 =====")

pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', XGBRegressor(random_state=42, eval_metric='rmse')) # eval_metric для предотвращения предупреждений
])

# Гиперпараметры для настройки XGBoost
param_grid_xgb = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__max_depth': [3, 5, 7],
    'regressor__subsample': [0.7, 1.0], # Доля выборок для обучения деревьев
    'regressor__colsample_bytree': [0.7, 1.0] # Доля признаков для каждого дерева
}

grid_search_xgb = GridSearchCV(
    pipeline_xgb,
    param_grid_xgb,
    cv=5,
    scoring=scoring_metrics,
    refit='R2',
    n_jobs=-1,
    verbose=1
)

grid_search_xgb.fit(X_train_ic50, y_train_ic50)

print("Лучшие параметры для XGBoost:", grid_search_xgb.best_params_)
print("Лучший R2 на кросс-валидации:", grid_search_xgb.best_score_)

# Оценка на тестовой выборке
y_pred_xgb = grid_search_xgb.predict(X_test_ic50)
mse_xgb = mean_squared_error(y_test_ic50, y_pred_xgb)
r2_xgb = r2_score(y_test_ic50, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test_ic50, y_pred_xgb)

print("\nМетрики на тестовой выборке (XGBoost):")
print(f"MSE: {mse_xgb:.4f}")
print(f"MAE: {mae_xgb:.4f}")
print(f"R2: {r2_xgb:.4f}")


===== XGBoost Регрессор для IC50 =====
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Лучшие параметры для XGBoost: {'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.05, 'regressor__max_depth': 3, 'regressor__n_estimators': 200, 'regressor__subsample': 0.7}
Лучший R2 на кросс-валидации: 0.44381606754397246

Метрики на тестовой выборке (XGBoost):
MSE: 0.4456
MAE: 0.5267
R2: 0.4559


In [11]:
# Модель 5: CatBoost Regressor

print("\n===== CatBoost Регрессор для IC50 =====")

pipeline_cat = Pipeline([
    ('scaler', StandardScaler()), # CatBoost менее чувствителен к масштабированию, но для пайплайна оставляем
    ('regressor', CatBoostRegressor(random_state=42, verbose=0)) # verbose=0 отключает вывод логов CatBoost
])

# Гиперпараметры для настройки CatBoost
param_grid_cat = {
    'regressor__iterations': [100, 200, 300], # Количество итераций (деревьев)
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__depth': [4, 6, 8], # Глубина деревьев
    'regressor__l2_leaf_reg': [1, 3, 5] # L2 регуляризация
}

grid_search_cat = GridSearchCV(
    pipeline_cat,
    param_grid_cat,
    cv=5,
    scoring=scoring_metrics,
    refit='R2',
    n_jobs=-1,
    verbose=1
)

grid_search_cat.fit(X_train_ic50, y_train_ic50)

print("Лучшие параметры для CatBoost:", grid_search_cat.best_params_)
print("Лучший R2 на кросс-валидации:", grid_search_cat.best_score_)

# Оценка на тестовой выборке
y_pred_cat = grid_search_cat.predict(X_test_ic50)
mse_cat = mean_squared_error(y_test_ic50, y_pred_cat)
r2_cat = r2_score(y_test_ic50, y_pred_cat)
mae_cat = mean_absolute_error(y_test_ic50, y_pred_cat)

print("\nМетрики на тестовой выборке (CatBoost):")
print(f"MSE: {mse_cat:.4f}")
print(f"MAE: {mae_cat:.4f}")
print(f"R2: {r2_cat:.4f}")


===== CatBoost Регрессор для IC50 =====
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Лучшие параметры для CatBoost: {'regressor__depth': 6, 'regressor__iterations': 200, 'regressor__l2_leaf_reg': 1, 'regressor__learning_rate': 0.05}
Лучший R2 на кросс-валидации: 0.45300200040076816

Метрики на тестовой выборке (CatBoost):
MSE: 0.4696
MAE: 0.5395
R2: 0.4266


In [12]:
# Модель 6: Простая Нейронная Сеть (Keras Sequential)
print("\n===== Простая Нейронная Сеть для IC50 =====")

from tensorflow import keras
from tensorflow.keras import layers as L
from scikeras.wrappers import KerasRegressor

# Функция для создания Keras-модели
# Параметры, которые мы хотим усовершенствовать через GridSearchCV, должны быть аргументами этой функции.
def build_nn_model(meta, hidden_layers=1, neurons=32, activation='relu',
                   optimizer='adam', learning_rate=0.001):
    n_features = meta["n_features_in_"]
    
    model = keras.Sequential()
    model.add(L.Input(shape=(n_features,)))
    
    for _ in range(hidden_layers):
        model.add(L.Dense(neurons, activation=activation))
        
    model.add(L.Dense(1)) # Выходной слой для регрессии (1 нейрон, без активации)
    
    # Создаем экземпляр оптимизатора с указанной скоростью обучения
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = keras.optimizers.Adam(learning_rate=learning_rate) # По умолчанию Adam

    model.compile(optimizer=opt, loss='mean_squared_error')
    return model

# Здесь мы указываем KerasRegressor
pipeline_nn = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', KerasRegressor(
        model=build_nn_model,
        # Задаем дефолтные значения для KerasRegressor, они будут переопределяться GridSearchCV
        # Важно: эти параметры должны быть аргументами build_nn_model
        hidden_layers=1,
        neurons=32,
        activation='relu',
        optimizer='adam',
        learning_rate=0.001,
        batch_size=32, # batch_size и epochs - это параметры .fit(), а не .build_model()
        epochs=50,
        verbose=0,
        random_state=42
    ))
])

# Гиперпараметры для настройки Нейронной Сети
# Параметры указываются с префиксом 'regressor__' для шага 'regressor' в пайплайне
# Эти параметры будут переданы в конструктор KerasRegressor, который затем передаст их в build_nn_model.
param_grid_nn = {
    'regressor__hidden_layers': [1, 2],
    'regressor__neurons': [32, 64],
    'regressor__activation': ['relu'],
    'regressor__optimizer': ['adam'],
    'regressor__learning_rate': [0.001, 0.01],
    'regressor__batch_size': [32, 64],
    'regressor__epochs': [50, 100]
}

grid_search_nn = GridSearchCV(
    pipeline_nn,
    param_grid_nn,
    cv=3,
    scoring=scoring_metrics,
    refit='R2',
    n_jobs=-1,
    verbose=1
)

grid_search_nn.fit(X_train_ic50, y_train_ic50)

print("Лучшие параметры для Нейронной Сети:", grid_search_nn.best_params_)
print("Лучший R2 на кросс-валидации:", grid_search_nn.best_score_)

# Оценка на тестовой выборке
y_pred_nn = grid_search_nn.predict(X_test_ic50)
mse_nn = mean_squared_error(y_test_ic50, y_pred_nn)
r2_nn = r2_score(y_test_ic50, y_pred_nn)
mae_nn = mean_absolute_error(y_test_ic50, y_pred_nn)

print("\nМетрики на тестовой выборке (Нейронная Сеть):")
print(f"MSE: {mse_nn:.4f}")
print(f"MAE: {mae_nn:.4f}")
print(f"R2: {r2_nn:.4f}")


===== Простая Нейронная Сеть для IC50 =====
Fitting 3 folds for each of 32 candidates, totalling 96 fits


C:\Users\admin\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Лучшие параметры для Нейронной Сети: {'regressor__activation': 'relu', 'regressor__batch_size': 64, 'regressor__epochs': 50, 'regressor__hidden_layers': 2, 'regressor__learning_rate': 0.001, 'regressor__neurons': 32, 'regressor__optimizer': 'adam'}
Лучший R2 на кросс-валидации: 0.3641236628320949

Метрики на тестовой выборке (Нейронная Сеть):
MSE: 0.5917
MAE: 0.5947
R2: 0.2775


Анализ результатов регрессии для log_IC50

1. Линейная Регрессия
R2 на кросс-валидации: 0.2813
Метрики на тестовой выборке: MSE: 0.5935, MAE: 0.6009, R2: 0.2753
Модель линейной регрессии продемонстрировала наиболее низкие показатели R2 среди всех протестированных моделей, как на кросс-валидации, так и на тестовой выборке. Значение R2 на уровне 0.275 на тестовой выборке указывает на то, что линейная модель объясняет лишь около 27.5% дисперсии целевой переменной. Это свидетельствует о наличии значительных нелинейных зависимостей в данных, которые не могут быть адекватно захвачены простым линейным подходом.

2. Random Forest Регрессор
Лучшие параметры: {'regressor__max_features': 0.6, 'regressor__min_samples_leaf': 5, 'regressor__n_estimators': 200}
R2 на кросс-валидации: 0.4293
Метрики на тестовой выборке: MSE: 0.4616, MAE: 0.5325, R2: 0.4364
Модель Random Forest показала значительно лучшие результаты по сравнению с линейной регрессией. Значение R2 на тестовой выборке составило 0.436, что означает объяснение более 43% дисперсии log_IC50. Это демонстрирует способность ансамблевых методов на основе деревьев эффективно моделировать нелинейные отношения в данных. Показатели MAE также снизились, указывая на повышение точности предсказаний.

3. LightGBM Регрессор
Лучшие параметры: {'regressor__learning_rate': 0.01, 'regressor__n_estimators': 300, 'regressor__num_leaves': 31, 'regressor__reg_alpha': 0.5, 'regressor__reg_lambda': 0.1}
R2 на кросс-валидации: 0.4253
Метрики на тестовой выборке: MSE: 0.5112, MAE: 0.5697, R2: 0.3758
LightGBM продемонстрировал хорошие результаты, однако уступил Random Forest по показателю R2 на тестовой выборке, достигнув значения 0.376. Расхождение между R2 на кросс-валидации и тестовой выборке (0.425 против 0.376) может указывать на некоторое переобучение модели или её меньшую обобщающую способность на новых, ранее не виденных данных, по сравнению с другими ансамблевыми методами.

4. XGBoost Регрессор
Лучшие параметры: {'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.05, 'regressor__max_depth': 3, 'regressor__n_estimators': 200, 'regressor__subsample': 0.7}
R2 на кросс-валидации: 0.4438
Метрики на тестовой выборке: MSE: 0.4456, MAE: 0.5267, R2: 0.4559
XGBoost показал наилучшие результаты среди всех протестированных моделей для прогнозирования log_IC50. Его R2 на тестовой выборке составил 0.456, что является самым высоким показателем и свидетельствует об объяснении почти 45.6% дисперсии целевой переменной. Значение MAE также является одним из самых низких (0.527), что указывает на высокую точность предсказаний.

5. CatBoost Регрессор
Лучшие параметры: {'regressor__depth': 6, 'regressor__iterations': 200, 'regressor__l2_leaf_reg': 1, 'regressor__learning_rate': 0.05}
R2 на кросс-валидации: 0.4530
Метрики на тестовой выборке: MSE: 0.4696, MAE: 0.5395, R2: 0.4266
CatBoost также продемонстрировал высокие результаты, но на тестовой выборке незначительно уступил XGBoost, достигнув R2 в 0.427. Несмотря на более высокий R2 на кросс-валидации (0.453), производительность на независимой тестовой выборке оказалась чуть ниже, чем у XGBoost, что является типичным для оценок обобщающей способности модели.

6. Простая Нейронная Сеть
Лучшие параметры: {'regressor__activation': 'relu', 'regressor__batch_size': 64, 'regressor__epochs': 50, 'regressor__hidden_layers': 2, 'regressor__learning_rate': 0.001, 'regressor__neurons': 32, 'regressor__optimizer': 'adam'}
R2 на кросс-валидации: 0.3641
Метрики на тестовой выборке: MSE: 0.5917, MAE: 0.5947, R2: 0.2775
Простая нейронная сеть показала результаты, сравнимые с линейной регрессией, с R2 на тестовой выборке 0.277. Это указывает на то, что данная архитектура нейронной сети недостаточно сложна или обучена для эффективного улавливания сложных закономерностей в данных, необходимых для точного прогнозирования log_IC50.

### Общий вывод по прогнозированию log_IC50:
Для задачи прогнозирования log_IC50 XGBoost Регрессор является наиболее эффективной моделью, демонстрируя самый высокий показатель R2 на тестовой выборке (0.456). Модели Random Forest и CatBoost также показали сильные результаты, подтверждая, что ансамблевые методы на основе деревьев решений хорошо подходят для данной задачи. В то же время, линейная регрессия и простая нейронная сеть оказались менее эффективными. Полученные результаты по log_IC50 значительно лучше, чем для log_SI, что указывает на лучшую объяснительную способность текущего набора признаков для этой целевой переменной